In [1]:
import requests
import time

In [2]:
stats_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [3]:
try:
    data = requests.get(stats_url)
    data.raise_for_status()
    
except requests.exceptions.RequestException as e:
    print("Error in GET method:", e)

In [4]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(data.text)
standings_table = soup.select("table.stats_table")[0]
standings_table
all_anchors = standings_table.find_all("a")
team_links = [link for link in [link.get("href") for link in all_anchors] if "/squads" in link]
final_team_links = [f"https://fbref.com{l}" for l in team_links]

In [5]:
team_url = final_team_links[0]
data = requests.get(team_url)

In [6]:
import pandas as pd
import io
match_stats = pd.read_html(io.StringIO(data.text), match="Scores & Fixtures")

C:\Users\atato\AppData\Local\Temp\ipykernel_6212\2189798775.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [7]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and "all_comps/shooting" in l]

In [8]:
data = requests.get(f"https://fbref.com{links[0]}")
shooting_stats = pd.read_html(io.StringIO(data.text), match="Shooting")[0]
shooting_stats.columns = shooting_stats.columns.droplevel()

In [9]:
team_stats = match_stats[0].merge(shooting_stats[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [5]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
years = list(range(2024, 2022, -1))
all_matches = []

from bs4 import BeautifulSoup
import io
import time
import pandas as pd

for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    time.sleep(1)  # avoid getting blocked from site
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(io.StringIO(data.text), match="Scores & Fixtures")[0]
        time.sleep(1)  # avoid getting blocked from site
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(io.StringIO(data.text), match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        # for unavailable shooting statistics
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(2)  # avoid getting blocked from site

full_match_data = pd.concat(all_matches)
full_match_data
full_match_data.to_csv("matches.csv")
    